# load and explore data

In [ ]:
from data import store,raw_data

In [ ]:
store

# call a extractor

In [ ]:
from competition.extractors import Raw,Combine,SelectByClickTime
extractFile = 'first_two_day.db'
extractor = Combine([Raw(),SelectByClickTime([170000,190000])])

In [ ]:
from run_1_extract_features import extract_features
extract_features(extractor,extractFile,raw_data)

# design a extractor 

In [ ]:
from competition.extractors.Base import BaseExtractor
import pandas as pd
import numpy as np
import os

class AppCvr(BaseExtractor):
    def __init__(self,X,y):
        key = "appID"
        dfTrain = X.copy()
        dfTrain['label']=y.copy()
        self.dfCvr = dfTrain.groupby(key).apply(lambda df: np.mean(df["label"])).reset_index()
        self.dfCvr.columns=['appID','appCvr']

    def get_train(self,X,y,raw_data):
        return self._extract(X,y,raw_data)

    def get_test(self,X,y,raw_data):
        return X,y,raw_data

    def _extract(self,X,y,raw_data):
        newX = pd.merge(X,self.dfCvr,on='appID',how='left')
        return newX,y,raw_data

In [ ]:
# configuration
from competition.extractors import Wrapper,Combine
from extracted import load_extract_by_name

loadFile = 'first_two_day.db'

ext = load_extract_by_name(loadFile)
trX = ext['trX']
trY = ext['trY']
teX = ext['teX']

extractor = Combine([Wrapper(trX,trY,teX),AppCvr(trX,trY)])

In [ ]:
from data import store,raw_data
newtrX,newtrY,_ = extractor.get_train(trX,trY,raw_data)

In [ ]:
print newtrX.columns

In [ ]:
print newtrX.shape, trX.shape

# save and run the new extractor
- save extractor into file
- modify and save `configCustom.py`
- run `run_1_extract_features.py`

In [ ]:
%%writefile competition/extractors/AppCvr.py
# -*- coding:utf-8 -*-
from competition.extractors.Base import BaseExtractor
import pandas as pd
import numpy as np
import os

class AppCvr(BaseExtractor):
    def __init__(self,X,y):
        key = "appID"
        dfTrain = X.copy()
        dfTrain['label']=y.copy()
        self.dfCvr = dfTrain.groupby(key).apply(lambda df: np.mean(df["label"])).reset_index()
        self.dfCvr.columns=['appID','appCvr']

    def get_train(self,X,y,raw_data):
        return self._extract(X,y,raw_data)

    def get_test(self,X,y,raw_data):
        return self._extract(X,y,raw_data)

    def _extract(self,X,y,raw_data):
        newX = pd.merge(X,self.dfCvr,on='appID',how='left')
        return newX,y,raw_data

In [ ]:
# only run once
!echo "from AppCvr import AppCvr" >> competition/extractors/__init__.py
!cat competition/extractors/__init__.py

In [ ]:
!cat config1.py

In [ ]:
%%writefile config1_rc.py
# -*- coding:utf-8 -*-
from competition.extractors import *

extractDir = './_extracted/'
extractFile = 'first_two_day_cvr.db'

# extractor
import pandas as pd
import os
loadFile = 'first_two_day.db'
ext = pd.HDFStore(os.path.join(extractDir,loadFile))
trX = ext['trX']
trY = ext['trY']
teX = ext['teX']
extractor = Combine([Wrapper(trX,trY,teX),AppCvr(trX,trY)])

In [ ]:
%run run_1_extract_features.py

In [1]:
from extracted import loadFile

In [2]:
loadFile()

<class 'pandas.io.pytables.HDFStore'>
File path: ./_extracted/first_two_day_cvr.db
/teX            frame        (shape->[338489,28])
/trX            frame        (shape->[454544,28])
/trY            series       (shape->[454544])   